<a href="https://colab.research.google.com/github/Nobel712/Deep-learning-project/blob/main/LSTM_GRU_Bi_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,GRU,SimpleRNN,Dense
from tensorflow.keras.layers import Activation,Dropout,Embedding
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling1D,Conv1D,MaxPooling1D,Flatten,Bidirectional,SpatialDropout1D
from tensorflow.keras.preprocessing import sequence,text
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.python.keras.utils import np_utils
from sklearn import preprocessing,decomposition,model_selection,metrics,pipeline
%matplotlib
from plotly import graph_objs as  go
import plotly.express as px
import plotly.figure_factory as ff

Using matplotlib backend: agg


In [ ]:
tf.test.is_gpu_available()

True

In [ ]:
try:
  tpu=tf.distribute.cluster_resolver.TPUClusterResolver()
  print('Running on GPU',tpu.master())
except ValueError:
  tpu=None
if tpu:
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy=tf.distribute.experimental.TPUStrategy(tpu)
else:
  strategy=tf.distribute.get_strategy()
print(strategy.num_replicas_in_sync)

1


In [ ]:
 train=pd.read_csv('/content/train.csv')

In [ ]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
train['threat'].value_counts()

0    159093
1       478
Name: threat, dtype: int64

In [ ]:
# col=list(train.columns)
# for i in col:
#   print('columns Name is :',i,"-->", train.value_counts)

In [ ]:
train.drop(['severe_toxic','obscene','threat','insult','identity_hate'],axis=1,inplace=True)
train=train.loc[:12000,:]
train.shape

(12001, 3)

In [ ]:
train['comment_text'][0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [ ]:
train['comment_text'].apply(lambda x:len(str(x).split())).max()

1403

In [ ]:
def roc_auc(prdictions,target):
  fpr,tpr,thresholds=metrics.roc_curve(target,prdictions)
  roc_auc=metrics.auc(fpr,tpr)
  return roc_auc

In [ ]:
xtrain,xvalid,ytrain,yvalid=train_test_split(train.comment_text.values,train.toxic.values,
                                            stratify=train.toxic.values,
                                            random_state=42,
                                            test_size=0.2,shuffle=True)

In [ ]:
token=text.Tokenizer(num_words=None)
max_len=1500
token.fit_on_texts(list(xtrain)+list(xvalid))
xtrain_seq=token.texts_to_sequences(xtrain)
xvalid_seq=token.texts_to_sequences(xvalid)
xtrain_pad=sequence.pad_sequences(xtrain_seq,maxlen=max_len)
xvalid_pad=sequence.pad_sequences(xvalid_seq,maxlen=max_len)
word_index=token.word_index

In [ ]:
word_index

{'the': 1,
 'to': 2,
 'of': 3,
 'and': 4,
 'a': 5,
 'you': 6,
 'i': 7,
 'is': 8,
 'that': 9,
 'in': 10,
 'it': 11,
 'for': 12,
 'this': 13,
 'not': 14,
 'on': 15,
 'be': 16,
 'as': 17,
 'are': 18,
 'have': 19,
 'with': 20,
 'your': 21,
 'if': 22,
 'was': 23,
 'article': 24,
 'or': 25,
 'but': 26,
 'page': 27,
 'my': 28,
 'an': 29,
 'wikipedia': 30,
 'by': 31,
 'from': 32,
 'do': 33,
 'at': 34,
 'about': 35,
 'me': 36,
 'so': 37,
 'talk': 38,
 'can': 39,
 'what': 40,
 'there': 41,
 'all': 42,
 'has': 43,
 'no': 44,
 'will': 45,
 'one': 46,
 'would': 47,
 'like': 48,
 'please': 49,
 'he': 50,
 'just': 51,
 'they': 52,
 'any': 53,
 'which': 54,
 'been': 55,
 'more': 56,
 'other': 57,
 'we': 58,
 "don't": 59,
 'his': 60,
 'should': 61,
 'some': 62,
 'here': 63,
 'see': 64,
 'who': 65,
 'also': 66,
 'because': 67,
 'know': 68,
 'am': 69,
 'think': 70,
 "i'm": 71,
 'edit': 72,
 'how': 73,
 'up': 74,
 'why': 75,
 'out': 76,
 "it's": 77,
 'then': 78,
 'people': 79,
 'use': 80,
 'only': 81,
 'w

In [ ]:
xtrain_pad

array([[   0,    0,    0, ..., 1502,   38, 6072],
       [   0,    0,    0, ...,    3,  101,  548],
       [   0,    0,    0, ...,  359,   11,   38],
       ...,
       [   0,    0,    0, ...,    4,   45, 1576],
       [   0,    0,    0, ...,    9,   77, 5989],
       [   0,    0,    0, ...,  166,  202,  514]], dtype=int32)

In [ ]:
%time
with strategy.scope():
  model=Sequential()
  model.add(Embedding(len(word_index)+1,300,input_length=max_len))
  model.add(SimpleRNN(100))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 1500, 300)         13049100  
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 100)               40100     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 13,089,301
Trainable params: 13,089,301
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.python.ops.gen_array_ops import batch_matrix_band_part_eager_fallback
model.fit(xtrain_pad,ytrain,epochs=1,batch_size=64*strategy.num_replicas_in_sync)

150/150 [==============================] - 161s 1s/step - loss: 0.1434 - accuracy: 0.9484


In [ ]:
score=model.predict(xvalid_pad)

76/76 [==============================] - 8s 102ms/step


In [ ]:
print("Auc: %.2f%%"  %(roc_auc(score,yvalid)))

Auc: 0.78%


In [ ]:
score_model=[]
score_model.append({'Model:' 'SimpleRNN','Auc_soce :',roc_auc(score,yvalid)})

In [ ]:
score_model

[{0.7847448216608781, 'Auc_soce :', 'Model:SimpleRNN'}]

In [ ]:
embeddings_index={}
with open('/content/glove.6B.100d.txt')as f:
  for line in f.readlines():
     values=line.split()
     word=values[0]
     coefs=np.asarray(values[1:], dtype='float32')
     embeddings_index[word]=coefs

In [ ]:
embedding_matrix=np.zeros((len(word_index)+1,100))
for word, i in tqdm(word_index.items()):
  embedding_vector=embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i]=embedding_vector

100%|██████████| 43496/43496 [00:00<00:00, 409441.00it/s]


In [ ]:
%time
with strategy.scope():
  model=Sequential()
  model.add(Embedding(len(word_index)+1,100,weights=[embedding_matrix],
                      input_length=max_len,
                      trainable=False))
  model.add(LSTM(100,dropout=.3,recurrent_dropout=0.3))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.87 µs
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 1500, 100)         4349700   
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 4,430,201
Trainable params: 80,501
Non-trainable params: 4,349,700
_________________________________________________________________


In [ ]:
model.fit(xtrain_pad,ytrain,epochs=1,batch_size=64*strategy.num_replicas_in_sync)

150/150 [==============================] - 325s 2s/step - loss: 0.2413 - accuracy: 0.9135


In [ ]:
scores=model.predict(xvalid_pad)
print("Auc : %.2f%%" %(roc_auc(scores,yvalid)))

76/76 [==============================] - 28s 366ms/step
Auc : 0.92%


In [ ]:
%time
with strategy.scope():
  model=Sequential()
  model.add(Embedding(len(word_index)+1,100,weights=[embedding_matrix],
                      input_length=max_len,
                      trainable=False))
  model.add(SpatialDropout1D(0.3))
  model.add(GRU(300))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.63 µs
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 1500, 100)         4349700   
                                                                 
 spatial_dropout1d (SpatialD  (None, 1500, 100)        0         
 ropout1D)                                                       
                                                                 
 gru (GRU)                   (None, 300)               361800    
                                                                 
 dense_3 (Dense)             (None, 1)                 301       
                                                                 
Total params: 4,711,801
Trainable params: 362,101
Non-trainable params: 4,349,700
_________________________________________________________________


In [ ]:
model.fit(xtrain_pad,ytrain,epochs=1,batch_size=64*strategy.num_replicas_in_sync)

150/150 [==============================] - 23s 124ms/step - loss: 0.2411 - accuracy: 0.9190


In [ ]:
scores=model.predict(xvalid_pad)
print("Auc : %.2f%%" %(roc_auc(scores,yvalid)))

76/76 [==============================] - 3s 33ms/step
Auc : 0.94%


In [ ]:
%time
with strategy.scope():
  model=Sequential()
  model.add(Embedding(len(word_index)+1,100,weights=[embedding_matrix],
                      input_length=max_len,
                      trainable=False))
  model.add(Bidirectional(LSTM(100,dropout=.3,recurrent_dropout=0.3)))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 1500, 100)         4349700   
                                                                 
 bidirectional (Bidirectiona  (None, 200)              160800    
 l)                                                              
                                                                 
 dense_4 (Dense)             (None, 1)                 201       
                                                                 
Total params: 4,510,701
Trainable params: 161,001
Non-trainable params: 4,349,700
_________________________________________________________________


In [ ]:
model.fit(xtrain_pad,ytrain,epochs=1,batch_size=64*strategy.num_replicas_in_sync)

150/150 [==============================] - 587s 4s/step - loss: 0.2417 - accuracy: 0.9168
